# Learning based methods

This is a model which takes a collection of images and figures out how to align them using an unsupervised approach. It is based on a paper by de Vos, Beredensen, Viergever, Sokooti, Staring, Isgum: [https://arxiv.org/pdf/1809.06130.pdf]().

In [1]:
import glob
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC'
tf.Session(config = config)

C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4770058216442267484
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3003121664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15496288655892731463
physical_device_desc: "device: 0, name: GeForce GTX 970, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


In [2]:
def load_img_paths(target):
    '''
    Retrieve the full path of all images in the dataset
    '''
    return glob.glob(target + '/*.tif')

data_dir = r'../data'
original_data_dir = data_dir + ('/learning')
all_files = pd.DataFrame(load_img_paths(original_data_dir))
all_files = all_files[0].values.tolist()
all_files[:5]

['../data/learning\\Tp26_Y000_X000_040.tif',
 '../data/learning\\Tp26_Y000_X001_040.tif',
 '../data/learning\\Tp26_Y000_X002_040.tif',
 '../data/learning\\Tp26_Y000_X003_040.tif',
 '../data/learning\\Tp26_Y000_X004_040.tif']

In [3]:
train_paths = all_files

## Network

> A ConvNet design for affine image registration. The network analyzes pairs of fixed and moving images in separate pipelines. Ending each pipeline with global average pooling enables analysis of input images of different sizes, and allows concatenation with the fully connected layers that have a fixed number of nodes connected to 12 affne transformation parameter outputs.

> The two separate pipelines analyze input pairs of fixed and moving images and each consist of five alternating 3x3x3 convolution layers and 2x2x2 downsampling layers. The number of these layers may vary, depending on task complexity and input image size. The weights of the layers are shared between the two pipelines to limit the number of total parameters in the network.

> The Conv-Nets were initialized with Glorot's uniform distribution (Glorot and Bengio, 2010) and optimized with Adam.

> Subsequently, the network can be connected to a neural network work that will decode the relative orientations of the fixed and moving images and convert those to 12 affine transformation parameters: *three translation*, *three rotation*, *three scaling*, and *three shearing parameters*.

2D images -> Two translation, `x,y`, one rotation `theta`, two scaling `dx, dy`, two shearing `gx, gy` = 7 parameters

In [4]:
from keras.layers.core import Dense, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D, Concatenate
from keras.models import Sequential, Model

def dlir_layer(model, filters):
    '''
    alternating 3x3 convolution layers and 2x2 downsampling layers
    '''
    model.add(Conv2D(filters, (3,3), activation='relu', padding='same'))
    model.add(AveragePooling2D()) # default size is 2x2
    
def affine_pipeline(model):
    '''
    five alternating 3x3 convolution layers and 2x2 downsampling layers
    Ending each pipeline with global average pooling
    '''
    filters = 32
    dlir_layer(model, filters)
    dlir_layer(model, filters)
    dlir_layer(model, filters)
    dlir_layer(model, filters)
    model.add(Conv2D(filters, (3,3), activation='relu', padding='same'))
    model.add(GlobalAveragePooling2D())
    
def my_DLIR(input_shape):
    '''
    Implement DLIR architecture
    '''
    floating_pipeline = Sequential()
    floating_pipeline.add(Lambda(lambda x : x, input_shape=input_shape))
    affine_pipeline(floating_pipeline)
    
    moving_pipeline = Sequential()
    moving_pipeline.add(Lambda(lambda x : x, input_shape=input_shape))
    affine_pipeline(moving_pipeline)
    
    cat = Concatenate()([floating_pipeline.output, moving_pipeline.output])
    cat = Dense(2048, activation='relu')(cat)
    cat = Dense(7,   activation='linear')(cat)
    return Model([floating_pipeline.input, moving_pipeline.input], cat)


Using TensorFlow backend.


In [5]:
from keras.optimizers import Adam
from skimage.io import imread

def aspect_resize(newsize, shape):
    '''
    Given an integer and a shape, return a tuple with the longest side of the shape = newsize
    '''
    m = np.argmax(shape)
    if m == 0:
        return (newsize, int(shape[1] / (shape[0] / newsize)))
    return (int(shape[0] / (shape[1] / newsize)), newsize, 1)

orig_shape = imread(all_files[0]).shape
img_size = aspect_resize(150, orig_shape)
batch_size = 16

my_model = my_DLIR(img_size)
my_model.compile(loss='mse', optimizer=Adam(lr=1e-5))
my_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lambda_1_input (InputLayer)     (None, 114, 150, 1)  0                                            
__________________________________________________________________________________________________
lambda_2_input (InputLayer)     (None, 114, 150, 1)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 114, 150, 1)  0           lambda_1_input[0][0]             
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 114, 150, 1)  0           lambda_2_input[0][0]             
__________________________________________________________________________________________________
conv2d_1 (

In [16]:
from os import path
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import img_to_array, load_img
from keras.utils import Sequence
from skimage.transform import resize

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
def image_pair_coords(fname, ymin=0, ymax=7, xmin=0, xmax=14):
    '''
    get an image's pair.
    '''
    im_coords = path.split(fname)[-1] \
                        .split('.')[0]    \
                        .split('_')[1:3]
    y_part = int(im_coords[0].split('Y')[1])
    x_part = int(im_coords[1].split('X')[1])
    if y_part == ymin:
        y_part = ymin+1
    elif y_part == ymax:
        y_part = ymax-1
    else:
        y_part  = y_part+1 if np.random.random() > 0.5 else y_part-1

    if x_part == xmin:
        x_part = xmin+1
    elif x_part == xmax:
        x_part = xmax-1
    else:
        x_part = x_part+1 if np.random.random() > 0.5 else x_part-1

    ystr = str(y_part).rjust(2,'0')
    xstr = str(x_part).rjust(2,'0')
    fpath = path.split(fname)[:-1][0]
    next_fname = path.join(fpath, 'Tp26_Y0%s_X0%s_040.tif' % (ystr, xstr))
    return next_fname       
        
class DataGenerator(Sequence):
    '''
    Adapted from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html
    Allows for multiprocessing in the fit generator
    '''

    def __init__(self, train_set, val_set, batch_size, im_size):
        self.train, self.val = train_set, val_set
        self.batch_size = batch_size
        self.im_size = im_size

    def __len__(self):
        return int(np.ceil(len(self.train) / float(self.batch_size)))

    # Will output sequence of tuples (image, test) given a datapath
    def __getitem__(self, idx):
        X1 = np.zeros(shape=(batch_size, self.im_size[0], self.im_size[1], 1))
        X2 = np.zeros(shape=(batch_size, self.im_size[0], self.im_size[1], 1))
        y = np.zeros(shape=(batch_size, 7))
        batch = self.train[idx * self.batch_size:(idx + 1) * self.batch_size]
        for j,fname in enumerate(batch):
            # to speed this up preprocess the images so they aren't resized on the fly
            X1[j] = img_to_array(load_img(fname, target_size=self.im_size, grayscale=True))
            fname_moving = image_pair_coords(fname)
            X2[j] = img_to_array(load_img(fname_moving, target_size=self.im_size, grayscale=True))
        return ([X1, X2], y)

    
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

# create weights file if it doesn't exist for ModelCheckpoint
from os import mkdir
try: 
    mkdir('tmp')
except FileExistsError:
    print('tmp directory already exists')
    
# descriptive weight file naming
checkpointer = ModelCheckpoint(filepath=('tmp/weights-%d-%d.hdf5' % 
                                         (batch_size, img_size[0])), 
                               verbose=1, save_best_only=True)

# history function
history = LossHistory()

tmp directory already exists


In [17]:
steps_per_epoch  = int(len(train_paths) / batch_size)
training_generator = DataGenerator(train_paths, [], batch_size, img_size)

hist = my_model.fit_generator(training_generator,
    epochs=10,
    workers=3,
    verbose=2,
    callbacks=[history, checkpointer, early_stopping]
)

Epoch 1/10
 - 4s - loss: 4.6613
Epoch 2/10


C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\callbacks.py:435: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 0s - loss: 2.4835
Epoch 3/10
 - 0s - loss: 1.1141
Epoch 4/10
 - 0s - loss: 0.4471
Epoch 5/10
 - 0s - loss: 0.1193
Epoch 6/10
 - 0s - loss: 0.0227
Epoch 7/10
 - 0s - loss: 0.0044
Epoch 8/10
 - 0s - loss: 0.0079
Epoch 9/10
 - 0s - loss: 0.0085
Epoch 10/10
 - 0s - loss: 0.0054
